In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
import numpy as np

In [ ]:
timesData = pd.read_csv("dataframe/timesData.csv")
timesData.head()

In [ ]:
timesData.info()

In [ ]:
timesData['income']=pd.to_numeric(timesData['income'], errors='coerce')

In [ ]:
shanghaiData = pd.read_csv("dataframe/shanghaiData.csv")
shanghaiData.head()

In [ ]:
school_and_country = pd.read_csv("dataframe/school_and_country_table.csv")
school_and_country.head()

In [ ]:
cwurData = pd.read_csv("dataframe/cwurData.csv")
cwurData.head()

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
education_expenditure = pd.read_csv("dataframe/education_expenditure_supplementary_data.csv",engine='python')
education_expenditure.head()

1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframe

In [ ]:
most_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmax())]
most_recent_data.head()

In [ ]:
least_recent_data=timesData.loc[(timesData.groupby('university_name')['year'].idxmin())]
least_recent_data.head()

In [ ]:
##timesData[timesData['year']>=timesData['year'].max()]

In [ ]:
##timesData[timesData['university_name']=='Aalto University']

In [ ]:
#timesData.groupby('university_name').min()['year']


In [ ]:
#timesData.groupby('university_name').max()['year']

2. For each university, compute the improvement in "income" between the least recent and the most recent data points

In [ ]:
punto2=pd.merge(most_recent_data, least_recent_data, on='university_name')
punto2.head()

In [ ]:
punto2.columns

In [ ]:
##nuovo_punto2=punto2[punto2[['income_x','income_y']].notnull()]
##nuovo_punto2.head()

In [ ]:
##def calcola_perc(riga):
    ##return riga['income_x'] - riga['income_y']

In [ ]:
##nuovo_punto2['income_improvement']=nuovo_punto2.apply(calcola_perc, axis=1)
##nuovo_punto2.head()

In [ ]:
##nuovo_punto2['differenza']=nuovo_punto2['income_x']-nuovo_punto2['income_y']
##nuovo_punto2

In [ ]:
#nuovo_punto2[nuovo_punto2['income_y']-nuovo_punto2['income_x']]

In [ ]:
##lista=nuovo_punto2['income_x']
##lista


In [ ]:
##nuovo=lista.astype(float)
##nuovo

In [ ]:
def calcola_perc(riga):
    return riga['votes_first_vote']/ risultato[riga['state']] *100

In [ ]:
##nuovo_punto2['income_y']=nuovo_punto2['income_y'].to_numeric()

In [ ]:
##nuovo_punto2=np.array(list, dtype=float)

In [ ]:
punto2['income_improvement']=punto2['income_y']-punto2['income_x']
punto2.head()

3. Find the university with the largest increase computed in the previous point

In [ ]:
punto2.loc[punto2['income_improvement']>= punto2['income_improvement'].max()]['university_name'] #possiamo anche togliere l'ultima quadra


4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [ ]:
cwurData[cwurData['institution']=='Aarhus University']

In [ ]:
shanghaiData[shanghaiData['university_name']=='Aarhus University']

5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

In [ ]:
most_recent_data.head()

In [ ]:
most_recent_data['year']

In [ ]:
most_recent_datab

In [ ]:
rapporto=most_recent_data['female_male_ratio']
rapporto.head()

In [ ]:
dataframe=pd.DataFrame(rapporto)
dataframe.head()

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['female'] = most_recent_data['female_male_ratio'].str.extract(r'(^\d+)')
most_recent_data['male'] = most_recent_data['female_male_ratio'].str.extract(r'(\d+$)')
most_recent_data.head()

In [ ]:
most_recent_data['male']=pd.to_numeric(most_recent_data['male'], errors='coerce')
most_recent_data['female']=pd.to_numeric(most_recent_data['female'], errors='coerce')

In [ ]:
most_recent_data.info()

In [ ]:
most_recent_data['num_students'] = most_recent_data['num_students'].str.replace(',','')
most_recent_data.head()

In [ ]:
most_recent_data['num_students']=pd.to_numeric(most_recent_data['num_students'], errors='coerce')
most_recent_data.head()

In [ ]:
##def calc_numero(row):
    ##return most_recent_data['num_students']* most_recent_data[row]//100

In [ ]:
#most_recent_data['num_male']=most_recent_data['male'].apply(calc_numero, axis= 1)

In [ ]:
most_recent_data['num_male']= most_recent_data['male'] * most_recent_data['num_students'] // 100
most_recent_data.head()

In [ ]:
most_recent_data['num_fem']= most_recent_data['female'] * most_recent_data['num_students'] // 100
most_recent_data.head()

6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

In [ ]:
educational_attainment = pd.read_csv("dataframe/educational_attainment_supplementary_data.csv")
educational_attainment.head()

In [ ]:
#educational_attainment.dropna(educational_attainment,how="any", thresh=None, subset=(['country_name','series_name']), inplace=True)
#educational_attainment.head() 

In [ ]:
educ = educational_attainment[(educational_attainment['country_name'].notnull()) | (educational_attainment['series_name'].notnull())]
educ.head()

In [ ]:
#educational_attainment.dropna?


9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

In [ ]:
ed_pulito= educ.dropna(axis='columns', how='all')
ed_pulito.head()

In [ ]:
#ed_pulito[ed_pulito['1986'].notnull()]

In [ ]:
ed_pulito.transpose?

10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

11. In the times ranking, compute the number of times each university appears

12. Find the universities that appear at most twice in the times ranking.

13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).